## Autores: 
Blanco García, Gabriel: gabriel.blanco@cunef.edu   
Ferrín Meilá, Michelle: michelle.ferrin@cunef.edu

# Optimización de hiperparámetros: regresión logística  

Aquí optimizaremos los hiperparámetros de la logistica, que es nuestro modelo ganador. La optimización de hiperparáemtros consiste en probar múltiples combinaciones de hiperparámetros de un determinado modelo, y seleccionar la combinación que maximice una métrica determinada. Se trata de afinar el modelo, para conseguir sacar el máximo partido de él.

In [1]:
# Operaciones básicas
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import scikitplot as skplt
import pickle

# Herramientas para optimizar y vectorizar
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

# El modelo
from sklearn.linear_model import LogisticRegression

# Métricas
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score 
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# Nuestras funciones
%run ../src/operar_modelos.ipynb
%run ../src/plots_metricas.ipynb

## Carga de datos y modelo

Cargamos los conjuntos de datos

In [9]:
X_train = pd.read_pickle('../data/train/X_train.sav')
y_train = pd.read_pickle('../data/train/y_train.sav')

X_validation = pd.read_pickle('../data/validation/X_validation.sav')
y_validation = pd.read_pickle('../data/validation/y_validation.sav')

X_test = pd.read_pickle('../data/test/X_test.sav')
y_test = pd.read_pickle('../data/test/y_test.sav')

Cargamos el modelo ganador, el cual va a ser optimizado

In [3]:
regresion_logistica = cargar_modelo('../models/trained_models/classifiers/LogisticRegression().sav')

In [4]:
regresion_logistica

Pipeline(steps=[('vectorizador', TfidfVectorizer()),
                ('clasificador', LogisticRegression())])

Inicializamos el vectorizador que forma parte de nuestro modelo

In [8]:
vectorizador = TfidfVectorizer()

## Optimización

Primero hay que definir el espacio de búsqueda, que consite en los distintos hiperparámetros que queremos probar. Utilizamos un diccionario con la siguiente estructura:

In [6]:
hiperparametros = {
    
    # Hay que usar la sintaxis: 'nombre del modelo en el pipe__nombre del hiperparámetro'
    'clasificador__penalty': ['l1', 'l2', 'elasticnet'], # los métodos de regularización: lasso, ridge 
                                                         # y el enfoque mixto
    
    'clasificador__C': [0.001, 0.01, 0.1, 1, 10, 100], # distintos valores para el parámetro de regularización
    
    'clasificador__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], # los distintos métodos 
                                                                                # por los que se estiman los 
                                                                                # parámetros.
    
    'clasificador__max_iter': [100, 250, 500, 1000] # distintos limites de iteraciones 
}

La ligereza del modelo puede aprovecharse para exprimir al máximo la optimización de sus hiperparámetros y probar todas las combinaciones posibles con `GridSearchCV()` y una tabla de hiperparémtros generosa, como la que hemos definido. Además, para garantizar la fiabilidad de los resultados, aplicaremos validación cruzada, con 10 folds. La métrica que se utiliza para juzgar las combinaciones de hiperparámetrois es el __área bajo la curva roc__

In [24]:
# Inicialización del objeto de optimización
regresion_logistica_optimizada = GridSearchCV(regresion_logistica, # el modelo
                                              
                                              hiperparametros, # los hiperparámetros definidos
                                              
                                              scoring='roc_auc', # selección por auc, métrica a maximizar
                                              
                                              cv=10, # validación cruzada de 10 folds
                                              
                                              verbose=1, # informacion sobre el proceso
                                              
                                              n_jobs=4) # 4 nucleos del ordenador trabajando en paralelo

Optimizamos el modelo sobre los datos de entrenamiento, pero la comparación con el modelo sin optimizar se hará empleando el conjunto de validación, para evitar sesgar la comparación

In [25]:
np.random.seed(1234)
regresion_logistica_optimizada.fit(X_train, y_train) # empieza la optimización

Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   53.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  4.3min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 10.2min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 18.3min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed: 29.2min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed: 42.7min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed: 63.1min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed: 224.3min
[Parallel(n_jobs=4)]: Done 3600 out of 3600 | elapsed: 389.6min finished


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vectorizador', TfidfVectorizer()),
                                       ('clasificador', LogisticRegression())]),
             n_jobs=4,
             param_grid={'clasificador__C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'clasificador__max_iter': [100, 250, 500, 1000],
                         'clasificador__penalty': ['l1', 'l2', 'elasticnet'],
                         'clasificador__solver': ['newton-cg', 'lbfgs',
                                                  'liblinear', 'sag', 'saga']},
             scoring='roc_auc', verbose=1)

Guardamos el objeto con los hiperparámetros optimizados

In [26]:
guardar_modelo(regresion_logistica_optimizada, '../models/trained_models/classifiers/logit_optimizada.sav')

Modelo guardado


Calculamos las métricas de la mejor combinación de hiperparáemtros __sobre el conjunto de validación__ para compararlo con la versión sin optimizar

In [28]:
# Esta es la combinación ganadora por AUC
regresion_logistica_optimizada.best_params_

{'clasificador__C': 10,
 'clasificador__max_iter': 100,
 'clasificador__penalty': 'l2',
 'clasificador__solver': 'saga'}

El solver saga hace referencia al Stochastic Average Gradient Descent, en su versión ampliada para permitir la implementación de la penalización l1. En este caso, la penalización ganadorra es l2, también conocida como Ridge.

Comparamos ahora la logit con su versión optimizada. Calculamos las métricas habituales que hemos empleado a lo largo del trabajo (AUC, F1, coste del modelo...). El procedimiento es similar al del notebook de construicción de modelos

In [20]:
# Cargamos la versión sin optimizar 
regresion_logistica = cargar_modelo('../models/trained_models/classifiers/LogisticRegression().sav')

In [31]:
# Dicionario para iterar
modelos = {'Regresion sin optimizar': regresion_logistica,
           'Regresion optimizada': regresion_logistica_optimizada}

In [45]:
for modelo in modelos: # para cada elemento del diccionario de modelos...
    
    # Se extrae el calsificador acceciendo con en nombre del modelo, y
    clasificador = modelos[modelo]
    
    # Si el modelo es la versión sin optimizar, se calculan sus métricas sobre test
    if (modelo == 'Regresion sin optimizar'):
        
        y_pred = clasificador.predict(X_test)
        y_pred_proba = clasificador.predict_proba(X_test)
        
        accuracy = clasificador.score(X_test, y_test)
        
        f1 = f1_score(y_test, y_pred)
        
        auc = roc_auc_score(y_test, y_pred_proba[:,1])
    
        coste = coste_modelo(clasificador)
    
        falsos_positivos = tasa_falsos_positivos(clasificador) * 100
        
        print(modelo)
        print('Accuracy:', round(accuracy, 3))
        print('F1:', round(f1, 3))
        print('AUC', round(auc, 3))
        print('Tasa de falsos positivos:', round(falsos_positivos, 3), '%')
        if (coste < 1):
            print('Ganancias de', (-1)*coste, '€')
        else:
                print('Pérdidas de', coste, '€')
        print('--------------------------------')
    
    # En caso contrario, si el modelo es el optimizado, sus métricas
    # se calculan con los conjuntos de validación
    else:
        
        y_pred = clasificador.predict(X_validation)
        y_pred_proba = clasificador.predict_proba(X_validation)
        
        accuracy = clasificador.score(X_validation, y_validation)
        
        f1 = f1_score(y_validation, y_pred)
        
        auc = roc_auc_score(y_validation, y_pred_proba[:,1])
    
        coste = coste_modelo(clasificador)
    
        falsos_positivos = tasa_falsos_positivos(clasificador) * 100
        
        print(modelo)
        print('Accuracy:', round(accuracy, 3))
        print('F1:', round(f1, 3))
        print('AUC', round(auc, 3))
        print('Tasa de falsos positivos:', round(falsos_positivos, 3), '%')
        if (coste < 1):
            print('Ganancias de', (-1)*coste, '€')
        else:
                print('Pérdidas de', coste, '€')
        print('--------------------------------')
        

Regresion sin optimizar
Accuracy: 0.89
F1: 0.891
AUC 0.956
Tasa de falsos positivos: 12.268 %
Ganancias de 441860 €
--------------------------------
Regresion optimizada
Accuracy: 0.96
F1: 0.897
AUC 0.96
Tasa de falsos positivos: 11.151 %
Ganancias de 461790 €
--------------------------------


Las métricas muestran que la optimización ha sido exitosa. Todas las métricas han mejorado. Los cambios más notables son en accuracy y en tasa de falsos positivos. Se han conseguido mejorar todas las métricas, aún reduciendo la tasa de falsos positivos, en más de un 1%. Este dato podría parecer nimio, pero con los costes que se han definido para este caso de uso, el resultado son casi 20.000 € de ganancia adicional. En términos unitarios (10.000 predicciones), la mejora es de 2 € por predicción, pasando de unos ingresos unitarios de 44.18 € a 46.18 €, __un incremento en los ingresos del modelo de un 4.53%__.

En síntesis, el modelo no solo supera a su versión básica en rendimiento técnico, sino también en genración de valor para la empresa, que es lo más importante.

En el siguiente notebook se calculan todas las métricas del modelo optimizado y se explican los resultados del mismo.